In [2]:
import pandas as pd
import numpy as np
import os
from agents import Runner, agent, trace, set_trace_processors, Agent, ModelSettings
from weave.integrations.openai_agents.openai_agents import WeaveTracingProcessor
import asyncio
from typing import Literal, List, Annotated
import weave
from dotenv import load_dotenv
from pydantic import BaseModel, Field, conlist
from IPython.display import display, Markdown, HTML, Image
from tqdm import tqdm
import openai
load_dotenv()

True

## Upload the file to the File API

In [4]:
import requests
from io import BytesIO
from openai import OpenAI

client = OpenAI()

def create_file(client, file_path):
    if file_path.startswith("http://") or file_path.startswith("https://"):
        # Download the file content from the URL
        response = requests.get(file_path)
        file_content = BytesIO(response.content)
        file_name = file_path.split("/")[-1]
        file_tuple = (file_name, file_content)
        result = client.files.create(
            file=file_tuple,
            purpose="assistants"
        )
    else:
        # Handle local file path
        with open(file_path, "rb") as file_content:
            result = client.files.create(
                file=file_content,
                purpose="assistants"
            )
    print(result.id)
    return result.id

# Replace with your own file path or URL
file_id = create_file(client, "../Data/Knowledge_Base/Workforce_paradigmshift_sid_Kimbho_etal.pdf")

file-KMjBJPHVCSCdh3F99LDh5Q


## Create a vector store

In [5]:
vector_store = client.vector_stores.create(
    name="knowledge_base"
)
print(vector_store.id)

vs_6897727443388191bb580a493335c756


## Add the file to the vector store

In [6]:
result = client.vector_stores.files.create(
    vector_store_id=vector_store.id,
    file_id=file_id
)
print(result)

VectorStoreFile(id='file-KMjBJPHVCSCdh3F99LDh5Q', created_at=1754755708, last_error=None, object='vector_store.file', status='in_progress', usage_bytes=0, vector_store_id='vs_6897727443388191bb580a493335c756', attributes={}, chunking_strategy=StaticFileChunkingStrategyObject(static=StaticFileChunkingStrategy(chunk_overlap_tokens=400, max_chunk_size_tokens=800), type='static'))


## Check Status

In [7]:
result = client.vector_stores.files.list(
    vector_store_id=vector_store.id
)
print(result)

PermissionDeniedError: Error code: 403 - {'error': "AI Gateway error: The endpoint 'api.openai.com/v1/vector_stores/vs_6897727443388191bb580a493335c756/files' you requested is not allowed. Make sure the url path is correct and is listed in the documentation (https://docs.prod.ai-gateway.quantumblack.com/models/). If you believe this is an error, please contact the AI Gateway team on #ai_gateway_users channel."}

In [10]:
from openai import OpenAI
client = OpenAI()

response = client.responses.create(
    model=os.getenv("LLM_MODEL"),
    input="What is this document about?",
    tools=[{
        "type": "file_search",
        "max_num_results": 20,
        "vector_store_ids": ["vs_6897727443388191bb580a493335c756"]
    }]
)
print(response)

Response(id='resp_6897733da01881a090ec11462a20732e00e0578b22e22a38', created_at=1754755901.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-5-2025-08-07', object='response', output=[ResponseReasoningItem(id='rs_6897733df9ec81a0bbedf62ffb4965ef00e0578b22e22a38', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFileSearchToolCall(id='fs_689773422fcc81a0b53b116eaa9e982000e0578b22e22a38', queries=['What is this document about?', 'executive summary overview introduction purpose', 'abstract summary key points', 'objective scope background of the document', 'summary of the uploaded document'], status='completed', type='file_search_call', results=None), ResponseReasoningItem(id='rs_68977344f71481a09cdfe9af8a54c75100e0578b22e22a38', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseOutputMessage(id='msg_6897734e12a481a09018378b542f1a2600e0578b22e22a38', content=[ResponseOutputText(